In [2]:
import numpy as np
import pandas as pd

In [3]:
rc = pd.read_csv('../data/rc.csv', parse_dates=['mp_start_time'])

In [6]:
# inspect entry

print(rc.shape)
print(rc.iloc[0])

(1724, 19)
left_to_pin_before                         144
penalty                                  False
hole_no                                     14
shot_no                                      3
completed                                 True
putt                                      True
left_to_pin_after                            1
par                                          4
yards                                      448
mp_name               Friday Foursomes Match 4
usa_player1                        Matt Kuchar
usa_player2                     Dustin Johnson
eur_player1                       Lee Westwood
eur_player2                     Thomas Pieters
mp_start_time              2016-09-30 13:20:00
player_shot_name                   Matt Kuchar
shot_team                                  USA
lie_after                                 Hole
lie_before                               Green
Name: 0, dtype: object


In [7]:
df = rc.copy()

# add cols to uniquely identity holes, shots

df['start_epoch'] = df['mp_start_time'].apply(lambda x: x.value // 10 ** 9)

def create_hole_id(r):
    fmt, player, team = r.mp_name, r.player_shot_name, r.shot_team
    name = (team if 'Foursomes' in fmt else player.replace(' ', '')).lower()

    return '{}_{}_{}'.format(r.start_epoch, name, r.hole_no)

df['hole_id'] = df.apply(create_hole_id, axis=1)
df['shot_id'] = df.apply(lambda r: '{}_{}'.format(r.hole_id, r.shot_no), axis=1)


# add score column

scores = df.groupby('hole_id')['shot_no'].agg({ 'score': max }).reset_index()
df = pd.merge(df, scores, on='hole_id')


# add event category

df['event_type'] = df['mp_name'].apply(lambda x: x.split(' ')[1])


# remove entries without left_to_pin_before/after info

df.dropna(subset=['left_to_pin_before', 'left_to_pin_after'], how='any', inplace=True)


# sort data (by time)

df.sort_values(by=['mp_start_time', 'hole_no', 'shot_no'], ascending=[1, 1, 1], inplace=True)
df.reset_index(inplace=True, drop=True)


# convert distance to pin to feet, yards, bins, etc.

def bin_num(x, base):
    if pd.isnull(x):
        return x
    return int(x - (x % base))

df['pre_ft'] = round(df['left_to_pin_before'] / 12.0).astype(int)
df['pre_yd'] = round(df['left_to_pin_before'] / 36.0).astype(int)
df['pre_ft_bin5'] = df['pre_ft'].apply(lambda x: bin_num(x, 5))
df['pre_yd_bin10'] = df['pre_yd'].apply(lambda x: bin_num(x, 10))
df['pre_yd_bin25'] = df['pre_yd'].apply(lambda x: bin_num(x, 25))

df['post_ft'] = round(df['left_to_pin_after'] / 12.0).astype(int)
df['post_yd'] = round(df['left_to_pin_after'] / 36.0).astype(int)
df['post_ft_bin5'] = df['post_ft'].apply(lambda x: bin_num(x, 5))
df['post_yd_bin10'] = df['post_yd'].apply(lambda x: bin_num(x, 10))
df['post_yd_bin25'] = df['post_yd'].apply(lambda x: bin_num(x, 25))


# assume final putts within 8 feet without a lie_after would have gone in

df['lie_after_adj'] = df['lie_after']
df.loc[(
    (pd.isnull(df.lie_after)) &
    (df.putt) &
    (df.pre_ft <= 8) &
    (df.shot_no == df.score)
), 'lie_after_adj'] = 'Hole'


# view data

print(df.shape)
df.iloc[0]

(1699, 35)


left_to_pin_before                       15912
penalty                                  False
hole_no                                      1
shot_no                                      1
completed                                 True
putt                                     False
left_to_pin_after                         5076
par                                          4
yards                                      442
mp_name               Friday Foursomes Match 1
usa_player1                       Patrick Reed
usa_player2                      Jordan Spieth
eur_player1                        Justin Rose
eur_player2                     Henrik Stenson
mp_start_time              2016-09-30 12:35:00
player_shot_name                  Patrick Reed
shot_team                                  USA
lie_after                              Fairway
lie_before                                 Tee
start_epoch                         1475238900
hole_id                       1475238900_usa_1
shot_id      

In [8]:
# check how many completed (adj) holes we have

completed = df.groupby('hole_id')['lie_after_adj'] \
    .agg({ 'did_complete': lambda x: sum(x == 'Hole') }) \
    .reset_index()

completed.did_complete.value_counts()

1    405
0     46
Name: did_complete, dtype: int64

In [9]:
# remove incomplete holes from dataset

dg = pd.merge(df, completed, on='hole_id')
dg = dg.query('did_complete == 1').copy()
len(dg)

1539

In [10]:
# categorize shots (to facilitate grouping and averaging # shots to complete hole)

def categorize(is_putt, ft, ft5, yds25, is_done=False):
    if is_done:
        return 'done'
    elif is_putt:
        return 'p_{}_ft'.format(ft if ft < 10 else '{}_to_{}'.format(ft5, ft5 + 5))
    else:
        return 'o_{}_to_{}_yds'.format(yds25, yds25 + 25)

dg['shots_left'] = dg['score'] - dg['shot_no'] + 1
dg['category'] = dg.apply(lambda r: categorize(r.putt, r.pre_ft, r.pre_ft_bin5, r.pre_yd_bin25), axis=1)
dg['category_next'] = dg.apply(
    lambda r: categorize(r.lie_after == 'Green', r.post_ft, r.post_ft_bin5, r.post_yd_bin25, r.shot_no == r.score),
    axis=1
)

dg.head(10)

,left_to_pin_before,penalty,hole_no,shot_no,completed,putt,left_to_pin_after,par,yards,mp_name,...,post_ft,post_yd,post_ft_bin5,post_yd_bin10,post_yd_bin25,lie_after_adj,did_complete,shots_left,category,category_next
0,15912.0,False,1,1,True,False,5076.0,4,442,Friday Foursomes Match 1,...,423,141,420,140,125,Fairway,1,4,o_425_to_450_yds,o_125_to_150_yds
1,5076.0,False,1,2,True,False,180.0,4,442,Friday Foursomes Match 1,...,15,5,15,0,0,Green,1,3,o_125_to_150_yds,p_15_to_20_ft
2,180.0,False,1,3,True,True,12.0,4,442,Friday Foursomes Match 1,...,1,0,0,0,0,Green,1,2,p_15_to_20_ft,p_1_ft
3,12.0,False,1,4,True,True,1.0,4,442,Friday Foursomes Match 1,...,0,0,0,0,0,Hole,1,1,p_1_ft,done
4,15444.0,False,2,1,True,False,4860.0,4,429,Friday Foursomes Match 1,...,405,135,405,130,125,Fairway,1,3,o_425_to_450_yds,o_125_to_150_yds
5,4860.0,False,2,2,True,False,96.0,4,429,Friday Foursomes Match 1,...,8,3,5,0,0,Green,1,2,o_125_to_150_yds,p_8_ft
6,96.0,False,2,3,True,True,1.0,4,429,Friday Foursomes Match 1,...,0,0,0,0,0,Hole,1,1,p_8_ft,done
7,22788.0,False,3,1,True,False,10728.0,5,633,Friday Foursomes Match 1,...,894,298,890,290,275,Fairway,1,4,o_625_to_650_yds,o_275_to_300_yds
8,10728.0,False,3,2,True,False,1296.0,5,633,Friday Foursomes Match 1,...,108,36,105,30,25,Intermediate Rough,1,3,o_275_to_300_yds,o_25_to_50_yds
9,1296.0,False,3,3,True,False,48.0,5,633,Friday Foursomes Match 1,...,4,1,0,0,0,Green,1,2,o_25_to_50_yds,p_4_ft


In [11]:
# compute average # of shots left, grouped by category
# (this is the baseline for shots gained stat) 

baseline = dg.groupby('category')['shots_left'].agg({ 'n': len, 'avg': np.mean })

# convert to dict
baseline_lookup = baseline.to_dict()['avg']
baseline_lookup['done'] = 0

baseline.sort_values(by='n', ascending=False).head()

,n,avg
category,,
p_1_ft,132,1.000000
o_0_to_25_yds,88,2.170455
p_2_ft,83,1.036145
o_425_to_450_yds,82,3.780488
o_125_to_150_yds,79,2.797468


In [12]:
# add baseline #s to data, compute strokes gained per shot

dg['baseline'] = dg['category'].apply(lambda x: baseline_lookup[x])
dg['baseline_next'] = dg['category_next'].apply(lambda x: baseline_lookup[x])
dg['strokes_gained'] = dg['baseline'] - dg['baseline_next'] - 1

In [13]:
# what were the best shots

cols = [
    'hole_no', 'shot_no', 'score', 'par',
    'mp_name', 'player_shot_name',
    'pre_yd', 'pre_ft', 'post_yd', 'post_ft',
    'lie_before', 'category', 'category_next',
    'baseline', 'baseline_next', 'strokes_gained'
]

dg.sort_values(by='strokes_gained', ascending=False)[cols].head(5)

,hole_no,shot_no,score,par,mp_name,player_shot_name,pre_yd,pre_ft,post_yd,post_ft,lie_before,category,category_next,baseline,baseline_next,strokes_gained
1382,6,3,3,5,Saturday Fourballs Match 4,Patrick Reed,133,399,0,0,Fairway,o_125_to_150_yds,done,2.797468,0.000000,1.797468
1488,8,2,2,3,Sunday Singles Match 1,Patrick Reed,7,21,0,0,Green Fringe,o_0_to_25_yds,done,2.170455,0.000000,1.170455
92,9,3,3,4,Friday Foursomes Match 2,Rickie Fowler,20,60,0,0,Green Fringe,o_0_to_25_yds,done,2.170455,0.000000,1.170455
911,5,3,3,4,Saturday Foursomes Match 4,Patrick Reed,6,18,0,0,Rough,o_0_to_25_yds,done,2.170455,0.000000,1.170455
1478,5,1,2,4,Sunday Singles Match 1,Patrick Reed,303,909,3,8,Tee,o_300_to_325_yds,p_8_ft,3.432432,1.416667,1.015766


**Videos of top 5 shots**

* [#1 Patrick Reed](http://media.snappytv.com/video/3128000/500p500/2016-10-01T20-01-20.1Z--31.3.mp4?token=1506913389_8faa5522f34e7479e0d7e9714f1b0cfd)
* [#2 Patrick Reed](http://media.snappytv.com/video/3128000/213p213/2016-10-02T17-39-11.367Z--32.2.mp4?token=1506990905_f1233df5b1202fb1d2358a3a806fe0bf)
* [#3 Rickie Fowler](http://media.snappytv.com/video/3128000/302p302/2016-09-30T14-46-13.1Z--34.433.mp4?token=1506809581_34525592b58d60ca44e16fbd542c4114)
* [#4 Patrick Reed](http://media.snappytv.com/video/3128000/500p500/2016-10-01T14-22-12.067Z--30.966.mp4?token=1506893464_e8213132b3fb1c465a6decf4a7f4e399)
* [#5 Patrick Reed](https://www.facebook.com/PGAcom/videos/10153922251002322/)


In [14]:
# compare total strokes gained by player and event format

dg.groupby(['player_shot_name', 'event_type']).agg({ 'strokes_gained': sum }).unstack()

strokes_gained                    
event_type            Fourballs Foursomes   Singles
player_shot_name                                   
Brandt Snedeker        1.902563  0.596690       NaN
Brooks Koepka          3.339887  0.135544  1.667643
Dustin Johnson         2.005072  2.780835       NaN
J B Holmes             0.944912       NaN       NaN
Jimmy Walker                NaN -0.656288 -0.934409
Jordan Spieth         -1.692885 -0.049520       NaN
Matt Kuchar           -1.113108 -1.738017       NaN
Patrick Reed           6.285709  2.003717  0.521524
Phil Mickelson         3.522941 -1.347770       NaN
Rickie Fowler               NaN -2.836611       NaN
Ryan Moore            -3.289823       NaN       NaN
Zach Johnson                NaN -3.068259  0.143833